In [1]:
## First, let's load and prepare the dataset
!pwd

/Users/ningning/Desktop/DL_Tasks/P2


In [21]:
# Import the required libraries
import os
from tensorflow.keras.utils import load_img, img_to_array
folders = os.listdir('Train')
# Uncommment the below statement to view the folders
# print(folders)

image_data = []
labels = []
count = 0

for ix in folders:
    path = os.path.join("Train", ix)
    # Uncommment the below statement to view the path to the images and their labels
    # print(path, count) 
    for im in os.listdir(path):
        try:
            img = load_img(os.path.join(path,im), target_size = (224,224))
            img_array = img_to_array(img)
            image_data.append(img_array)
            labels.append(count)
        except:
            print("error")
            pass
    count += 1

error
error


In [22]:
count

10

In [23]:
path

'Train/Dratini'

In [24]:
## shuffle the dataset

import random

combined_dataset = list(zip(image_data, labels))
random.shuffle(combined_dataset)
image_data[:], labels[:] = zip(*combined_dataset)

In [27]:
from keras.utils import np_utils
import numpy as np
X_train = np.array(image_data)
Y_train = np.array(labels)

Y_train = np_utils.to_categorical(Y_train)

In [34]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout,GlobalAveragePooling2D
from tensorflow.keras.models import Model
print("Imported Successfully!")

Imported Successfully!


# Assignment 1

Change the hyperparameters, the number of layers in our classifier, and the number of layers up to which we want to freeze the weight update process and see what the accuracy of the model is. How does it change?

In [32]:
model = ResNet50(include_top = False, weights = 'imagenet', input_shape = (224,224,3))
print(model.summary())

94765736/94765736 [==============================] - 6s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [35]:
# The ResNet50 model's output is going to be connected to this classifier.

av1 = GlobalAveragePooling2D()(model.output)

fc1 = Dense(256, activation = 'relu')(av1)

d1 = Dropout(0.5)(fc1)

fc2 = Dense(128, activation = 'relu')(d1)

d2 = Dropout(0.5)(fc2)

fc3 = Dense(64, activation = 'relu')(d2)

d3 = Dropout(0.5)(fc3)

fc4 = Dense(10, activation = 'softmax')(d3)

In [36]:
model_new = Model(inputs = model.input, outputs = fc4)
model_new.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [41]:
!pwd

/Users/ningning/Desktop/DL_Tasks/P2


In [43]:
# from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
image_path = 'Train/pikachiu.jpg'
img = load_img(image_path,target_size = (224,224))
x = img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)

pred = model_new.predict(x)
print(np.argmax(pred))

2024-10-06 20:50:58.639766: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 1s 711ms/step
0


In [44]:
adam = Adam(lr = 0.00003)
model_new.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

In [45]:
for ix in range(len(model_new.layers)):
    print(ix, model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x17719e2d0>
1 <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x29582e890>
2 <keras.layers.convolutional.conv2d.Conv2D object at 0x173fe6290>
3 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x29599f0d0>
4 <keras.layers.core.activation.Activation object at 0x295805550>
5 <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x1771e1e90>
6 <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x2958c8750>
7 <keras.layers.convolutional.conv2d.Conv2D object at 0x1774c8250>
8 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x2959f2550>
9 <keras.layers.core.activation.Activation object at 0x29574f690>
10 <keras.layers.convolutional.conv2d.Conv2D object at 0x295a20610>
11 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x29594c350>
12 <keras.layers.core.activation.Activation object at 0x2958c9b90>
13 <keras.layers.convolutional

In [46]:
for ix in range(169):
    model_new.layers[ix].trainable = False
print(model_new.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [47]:
hist = model_new.fit(X_train, Y_train, shuffle = True, batch_size = 16, epochs = 3, validation_split = 0.20)

Epoch 1/3
83/83 [==============================] - 278s 3s/step - loss: 2.6444 - accuracy: 0.1641 - val_loss: 25616.2988 - val_accuracy: 0.1915
Epoch 2/3
83/83 [==============================] - 191s 2s/step - loss: 2.1968 - accuracy: 0.2287 - val_loss: 83.5146 - val_accuracy: 0.2614
Epoch 3/3
83/83 [==============================] - 189s 2s/step - loss: 2.0679 - accuracy: 0.3108 - val_loss: 6.2368 - val_accuracy: 0.1763


# Problem 2
We used the ResNet50 model here. Your task is to use the MobileNet and AlexNet pre-trained model and build a classifier on the same dataset. You may also use any other image classification dataset.

In [49]:
from tensorflow.keras.applications import MobileNet

In [50]:
# Load the MobileNet model
mobile_net_model = MobileNet(include_top=False, weights='imagenet', input_shape = (224,224,3))
mobile_net_model.summary()

17225924/17225924 [==============================] - 1s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32

In [51]:
av1 = GlobalAveragePooling2D()(mobile_net_model.output)
fc1 = Dense(256, activation = 'relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(128, activation = 'relu')(d1)
d2 = Dropout(0.5)(fc2)
fc3 = Dense(64, activation = 'relu')(d2)
d3 = Dropout(0.5)(fc3)
fc4 = Dense(10, activation = 'softmax')(d3)

In [52]:
mobile_net_final_model = Model(inputs = mobile_net_model.input, outputs = fc4)
mobile_net_final_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [53]:
adam = Adam(lr = 0.00003)
mobile_net_final_model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

# Freeze the training process for the first 50 layers of MobileNet model
for ix in range(50):
    mobile_net_final_model.layers[ix].trainable = False
print(mobile_net_final_model.summary())

# Train the model
hist = mobile_net_final_model.fit(X_train, Y_train, shuffle = True, batch_size = 16, epochs = 3, validation_split = 0.20)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [57]:
! ls -l Train/Pikachu

total 126608
-rwxr--r--@ 1 ningning  staff    75804 10 Sep  2020 00000000.jpg
-rwxr--r--@ 1 ningning  staff    66441 10 Sep  2020 00000001.png
-rwxr--r--@ 1 ningning  staff   245087 10 Sep  2020 00000002.png
-rwxr--r--@ 1 ningning  staff    73232 10 Sep  2020 00000003.jpg
-rwxr--r--@ 1 ningning  staff   149804 10 Sep  2020 00000004.jpg
-rwxr--r--@ 1 ningning  staff   177165 10 Sep  2020 00000006.png
-rwxr--r--@ 1 ningning  staff   468924 10 Sep  2020 00000007.jpg
-rwxr--r--@ 1 ningning  staff    66913 10 Sep  2020 00000009.png
-rwxr--r--@ 1 ningning  staff    74314 10 Sep  2020 00000010.png
-rwxr--r--@ 1 ningning  staff   122285 10 Sep  2020 00000011.png
-rwxr--r--@ 1 ningning  staff   282099 10 Sep  2020 00000013.png
-rwxr--r--@ 1 ningning  staff    56578 10 Sep  2020 00000014.png
-rwxr--r--@ 1 ningning  staff   844366 10 Sep  2020 00000015.png
-rwxr--r--@ 1 ningning  staff    99548 10 Sep  2020 00000016.jpg
-rwxr--r--@ 1 ningning  staff   160790 10 Sep  2020 00000017.jpg
-rwxr--r--@ 

In [58]:
from tensorflow.keras.applications.mobilenet import preprocess_input

image_path = 'Train/Pikachu/00000098.jpg'
img = load_img(image_path,target_size = (224,224))
x = img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)

pred = mobile_net_final_model.predict(x)
print(np.argmax(pred))

1/1 [==============================] - 0s 193ms/step
0
